In [ ]:
import traceback
from kivy.app import App
from kivy.core.window import Window
from kivy.lang import Builder
from kivy.uix.screenmanager import Screen, ScreenManager
from kivy.factory import Factory
from kivy.animation import Animation
from kivy.uix.boxlayout import BoxLayout
from kivy.properties import ListProperty
from kivy.metrics import dp
from kivy.uix.button import Button
from kivy.uix.behaviors import ButtonBehavior
from kivymd.uix.behaviors import CircularRippleBehavior
from kivymd.toast.kivytoast.kivytoast import toast
from kivy.core.audio import SoundLoader
from kivy.uix.gridlayout import GridLayout
from kivy.uix.scrollview import ScrollView

from kivymd.app import MDApp
from kivymd.uix.bottomnavigation import MDBottomNavigationItem
from kivymd.uix.card import MDCard

import matplotlib.pyplot as plt
from kivy.uix.floatlayout import FloatLayout
from kivy.garden.matplotlib.backend_kivyagg import FigureCanvasKivyAgg
from kivy.uix.image import Image
from PIL import Image

from kivy.config import Config
Config.set('graphics', 'fullscreen','0')

import csv
from datetime import datetime

import cv2
import numpy as np
from os import makedirs
from os import listdir
from os.path import isdir, isfile, join

Window.softinput_mode = "below_target"  # resize to accomodate keyboard
Window.keyboard_anim_args = {'d': 0.5, 't': 'in_out_quart'}

Builder.load_string("""
#:import utils kivy.utils
#:include screen.kv
""")


class JobListScreen(Screen):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    
    def on_enter(self):
        for _ in range(10):
            MDApp.get_running_app().job_list.append({"height": dp(150)})


class JobListCard(MDCard):
    def prepare_viewing_of_publication(self):
        print(int(self.publication_id))
        
    def view_job(self, job_card):
        print(job_card)
    
    def toggle_heart(self, widget):
        if widget.icon == "heart":
            widget.icon = "heart-outline" if widget.icon == "heart" else "heart-outline"
            toast("Job unsaved")
        else:
            widget.icon = "heart" if widget.icon == "heart-outline" else "heart"
            toast("Job saved")


class HomeScreen(Screen):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def on_enter(self):
        MDApp.get_running_app().category_list.append({"height": dp(150)})
        MDApp.get_running_app().category_list.append({"height": dp(150)})
        MDApp.get_running_app().category_list.append({"height": dp(150)})
        MDApp.get_running_app().category_list.append({"height": dp(150)})
        MDApp.get_running_app().category_list.append({"height": dp(150)})

        
class CategoryCard(MDCard,CircularRippleBehavior, ButtonBehavior):
    def open_category(self, widget):
        print(widget)
        
    def on_release(self):
        MDApp.get_running_app().manage_screens("job_list_screen", "add")
        MDApp.get_running_app().change_screen("job_list_screen")
        
        
class MyLayout(BoxLayout):pass

# Adapter pattern 적용
class Show():
    def __init__(self, date):
        self.date = date
        
    def showDate(self):
        format = '%Y-%m-%d'
        str_date = datetime.strftime(self.date,format)
        return str_date
        
    def showTime(self):
        format = '%H:%M:%S'
        str_time = datetime.strftime(self.date,format)
        return str_time
        
class Date_info():
    def get_date(self):
        pass
    
    def get_time(self):
        pass
    
class ShowDate_info(Show,Date_info):
    def __init__(self,date):
        super(ShowDate_info, self).__init__(date)
        
    def get_date(self):
        return self.showDate()
        
    def get_time(self):
        return self.showTime()
    
class FaceExtractor:
    def __init__(self):
        #cascade classifier(다단계 분류)를 이용하여 객체 검출(object detection) -> 얼굴 검출
        self.face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    
    # 얼굴 검출 함수
    def face_extractor(self,img):
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        faces = self.face_classifier.detectMultiScale(gray,1.3,5)
        # 얼굴이 없으면 패스
        if faces is():
            return None
        # 얼굴이 있으면 얼굴 부위만 이미지로 만들고
        for(x,y,w,h) in faces:
            cropped_face = img[y:y+h, x:x+w]
        
        return cropped_face
    
class TakePictures:
    def __init__(self):
        # 얼굴 저장 경로
        self.face_dirs = 'faces/'
        self.FE = FaceExtractor()
    
    def make_dir(self,name):
        # 해당 이름의 폴더가 없다면 생성
        if not isdir(self.face_dirs+name):
            makedirs(self.face_dirs+name)
            
    def take_pictures(self,name):
        # 카메라 ON    
        cap = cv2.VideoCapture(0)
        count = 0

        while True:
            # 카메라로 부터 사진 한장 읽어 오기
            ret, frame = cap.read()
            # 사진에서 얼굴 검출 , 얼굴이 검출되었다면 
            if self.FE.face_extractor(frame) is not None:

                count+=1
                # 200 x 200 사이즈로 줄이거나 늘린다음
                face = cv2.resize(self.FE.face_extractor(frame),(200,200))
                # 흑백으로 바꿈
                face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

                # 200x200 흑백 사진을 faces/사용자이름/userxx.jpg 로 저장
                file_name_path = self.face_dirs + name + '/user'+str(count)+'.jpg'
                cv2.imwrite(file_name_path,face)

                cv2.putText(face,str(count),(50,50),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
                cv2.imshow('Face Cropper',face)
            else:
                print("Face not Found")
                pass

            # 얼굴 사진 100장을 다 얻었거나 enter키 누르면 종료
            if cv2.waitKey(1)==13 or count==100:
                break

        cap.release()
        cv2.destroyAllWindows()
        print('Colleting Samples Complete!!!')

class Train:
    def __init__(self):
        # LBF(Local Binary Pattern)을 이용하여 model을 수집한 100장의 training data로 학습
        self.model = cv2.face.LBPHFaceRecognizer_create()
        
    # 사용자 얼굴 학습
    def train(self, name):
        data_path = 'faces/' + name + '/'
        #파일만 리스트로 만듬
        face_pics = [f for f in listdir(data_path) if isfile(join(data_path,f))]

        Training_Data, Labels = [], []

        for i, files in enumerate(face_pics):
            image_path = data_path + face_pics[i]
            images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            # 이미지가 아니면 패스
            if images is None:
                continue    
            Training_Data.append(np.asarray(images, dtype=np.uint8))
            Labels.append(i)
        if len(Labels) == 0:
            print("There is no data to train.")
            return None
        Labels = np.asarray(Labels, dtype=np.int32)

        # 학습
        self.model.train(np.asarray(Training_Data), np.asarray(Labels))
        print(name + " : Model Training Complete!!!!!")

        #학습 모델 리턴
        return self.model
    
class Trains:
    def __init__(self):
        self.TR = Train()
        self.data_path = 'faces/'
    def trains(self):
        #faces 폴더의 하위 폴더를 학습, 폴더만 색출
        model_dirs = [f for f in listdir(self.data_path) if isdir(join(self.data_path,f))]

        #학습 모델 저장할 딕셔너리
        models = {}
        # 각 폴더에 있는 얼굴들 학습
        for model in model_dirs:
            print('model :' + model)
            # 학습 시작
            result = self.TR.train(model)
            # 학습이 안되었다면 패스
            if result is None:
                continue
            # 학습되었으면 저장
            print('model2 :' + model)
            models[model] = result

        # 학습된 모델 딕셔너리 리턴
        return models

class FaceDetector:
    def __init__(self):
        # cascade classifier(다단계 분류)를 이용하여 객체 검출(object detection) -> 얼굴 검출
        # 얼굴 인식용 haar/cascade 로딩
        self.face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
        
    #얼굴 검출
    def face_detector(self,img):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = self.face_classifier.detectMultiScale(gray,1.3,5)
        if faces is():
            return img,[]
        for(x,y,w,h) in faces:
            cv2.rectangle(img, (x,y),(x+w,y+h),(0,255,255),2)
            roi = img[y:y+h, x:x+w]
            roi = cv2.resize(roi, (200,200))
            
        return img,roi   #검출된 좌표에 사각 박스 그리고(img), 검출된 부위를 잘라(roi) 전달

class Recognition:
    def __init__(self):
        self.TRS = Trains()
        self.FD = FaceDetector()
        
    def recognition(self):
        #카메라 열기 
        cap = cv2.VideoCapture(cv2.CAP_DSHOW+0)

        while True:
            #카메라로 부터 사진 한장 읽기 
            ret, frame = cap.read()
            # 얼굴 검출 시도 
            image, face = self.FD.face_detector(frame)
            try:            
                min_score = 999       #가장 낮은 점수로 예측된 사람의 점수
                min_score_name = ""   #가장 높은 점수로 예측된 사람의 이름

                #검출된 사진을 흑백으로 변환 
                face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
                
                #위에서 학습한 모델로 예측시도
                for key, model in self.TRS.models.items():
                    result = model.predict(face)
                    if min_score > result[1]:
                        min_score = result[1]
                        min_score_name = key

                #min_score 신뢰도이고 0에 가까울수록 해당 인물과 같다 의미        
                if min_score < 500:
                    confidence = int(100*(1-(min_score)/300))
                    # 유사도 화면에 표시 
                    display_string = str(confidence)+'% Confidence it is ' + min_score_name
                cv2.putText(image,display_string,(100,120), cv2.FONT_HERSHEY_COMPLEX,1,(250,120,255),2)
                #75 보다 크면 등록인 -> Verified! 
                if confidence > 75:
                    cv2.putText(image, "Verified : " + min_score_name, (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
                    cv2.imshow('Face Cropper', image)
                    if cv2.waitKey(3000):
                        #기록 저장하기
                        now = datetime.now()
                        with open('check.txt', 'a', encoding='UTF-8', newline='') as f:
                            f.writelines(['\n',min_score_name,now.strftime(' %Y-%m-%d %H:%M:%S')])
                        f.close()
                    break
                else:
                #75 이하면 미등록인 -> Unverified
                    cv2.putText(image, "Unverified", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)
                    cv2.imshow('Face Cropper', image)
                    if cv2.waitKey(3000): break
            except:
                #얼굴 검출 안됨 
                cv2.putText(image, "Face Not Found", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0), 2)
                cv2.imshow('Face Cropper', image)
                pass

        cap.release()
        cv2.destroyAllWindows()
        
class Learning:
    def __init__(self):
        self.TP = TakePictures()
        self.TRS = Trains()
        self.RC = Recognition()
        
    def register(self,name):
        self.TP.make_dir(name)
        self.TP.take_pictures(name)
        
    def recognition(self):
        self.TRS.trains()
        self.RC.recognition()      

class Emp(MDApp):
    job_list = ListProperty() # contains data needed to display job list cards
    category_list = ListProperty() # contains data needed to display job list cards
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.title = "NUGU"
        self.theme_cls.primary_palette = "DeepPurple"
        self.theme_cls.theme_style = "Light"
        self.sm = ScreenManager()
        self.has_animated_card = False
        self.has_animated_background = False
    
    def account_action(self, email, passwd, username=None, action=None):
        print(email, passwd, username, action)
        if action == "register":
            pass
            # register the user
        elif action == "login":
            # login the user
            pass
        self.manage_screens("home_screen", "add")
        self.change_screen("home_screen")
    
    def animate_background(self, widget):
        if self.has_animated_background == False:
            anim = Animation(size_hint_y=1) + Animation(size_hint_y=0.5)
            anim.start(widget.ids.bx)
            print("background animated")
        else:
            print("background already animated")
    
    def animate_card(self, widget):
        if self.has_animated_card == False:
            anim = Animation(pos_hint={"center_x": 0.5, "center_y": 0.6}, duration=2)
            anim.start(widget)
            self.has_animated_card = True
            print("card animated")
        else:
            print("card already animated")
            
    #어플 화면을 변경할 때 사용하는 함수
    def change_screen(self, screen_name):
        if self.sm.has_screen(screen_name):
            self.sm.current = screen_name
            print("Change to [" + screen_name + "]. ")
        else:
            print("Screen [" + screen_name + "] does not exist.")

    def manage_screens(self, screen_name, action):
        scns = {
            "main_screen": Factory.LoginScreen,
            "check_visitor_screen": Factory.RegistrationScreen,
            "home_screen": Factory.HomeScreen,
            "job_list_screen": Factory.JobListScreen
        }
        try:

            if action == "remove":
                if self.sm.has_screen(screen_name):
                    self.sm.remove_widget(self.sm.get_screen(screen_name))
                print("Screen ["+screen_name+"] removed")
            elif action == "add":
                if self.sm.has_screen(screen_name):
                    print("Screen [" + screen_name + "] already exists")
                else:
                    self.sm.add_widget(scns[screen_name](name=screen_name))
                    print(screen_name + " added")
                    print("Screen ["+screen_name+"] added")
        except:
            print(traceback.format_exc())
            print("Traceback ^.^")

    def on_pause(self):
        return True

    def on_resume(self):
        pass
        
    def start_register(self):
        main_screen_instance = self.root.get_screen('main_screen')
        name = main_screen_instance.ids["name"].text
        learn = Learning()
        learn.register(name)
        
    def start_recognition(self):
        learn = Learning()
        learn.recognition()
        
    def getDate(self):
        sd = ShowDate_info(datetime.now())
        return sd.get_date()
    
    def getTime(self):
        sd = ShowDate_info(datetime.now())
        return sd.get_time()
    
    # kv파일의 check_visitor_screen에서는 text 타입만 보여줄 수 있는 특징이 있기 때문에
    # 방문기록이 저장된 텍스트 파일의 내용일 읽고 내용들을 하나의 string으로 만들어 방문기록을 보여줄 수 있도록 하였습니다.
    def get_line(self): #저장된 기록을 불러오는 함수
        b = ''
        f = open("check.txt", 'r')
        while True:
            line = f.readline()
            if not line: break
            # print(line)
            b += line
        f.close()
        b += '\n'*2
        return b

    def build(self):
        self.bind(on_start=self.post_build_init)
        self.sm.add_widget(Factory.LoginScreen())
        
        return self.sm

    def post_build_init(self, ev):
        win = self._app_window
        win.bind(on_keyboard=self._key_handler)

    def _key_handler(self, *args):
        key = args[1]
        # 1000 is "back" on Android
        # 27 is "escape" on computers
        if key in (1000, 27):
            try:
                self.sm.current_screen.dispatch("on_back_pressed")
            except Exception as e:
                print(e)
            return True
        elif key == 1001:
            try:
                self.sm.current_screen.dispatch("on_menu_pressed")
            except Exception as e:
                print(e)
            return True
        
    
if __name__ == "__main__":
    Emp().run()